In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [4]:

# # 含非自营
# sql = '''-- 租后应收监控  
# SELECT  tprm.* 
# ,om.`status` as 订单状态值
# ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
# when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
# when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
# ,om.order_finish_date as 订单完成时间,om.order_method
# ,tmu.true_name ,tmu.id_card_num,tmu.mobile 
# ,tod.product_name
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# left join db_digua_business.t_order om on tprm.order_id = om.id
# left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
# left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
# ;
# '''

# df_zhys = query(sql)
# # (27787, 62)
# df_zhys.shape

# note 2025.4.30
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile,tmu.province_name
,tod.product_name
,tod.sku_attributes

-- 2025.4.29
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(63691, 73)

## 1_12. 剔除商家数据

In [77]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(59809, 86)

## 1_13.历史订单渠道名称补充

In [78]:
# # 1148 条历史订单渠道名称补充
# qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# # dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
# dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# # dfzh1.columns

# # qudao_name_df.shape
# # qudao_name_df[:2]
# # qudao_name_df['渠道名称'].value_counts()

# # 渠道名称缺失补充
# # dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
# dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
# dfzh1["channel_name"].isnull().sum()

# # dfzh1[['order_number','channel_name','渠道名称']][:10]
# # df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]


# note 2025.4.30
# 1148 条历史订单渠道名称补充
# qudao_name_df = pd.read_excel('E:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys#.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
# dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name"])
# dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

# 更新渠道，同时给客群打标签
# 2025.4.29
# 是否巨量、是否号卡
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1["channel_name_cc"] = np.where(dfzh1.渠道名称.notnull(), dfzh1.渠道名称, dfzh1["channel_name_cc"])
dfzh1.loc[:,"来源渠道"]=dfzh1["channel_name"].fillna("未知渠道")
dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', dfzh1.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
dfzh1['归属渠道'] = np.where(dfzh1.归属渠道.isin(conditions_1), 'S量', dfzh1.归属渠道)


# dfzh1[dfzh1.归属渠道 == '顶部搜索框的搜索结果页']，channel_name 为 顶部搜索框的搜索结果页，channel_name_cc 为 空，下单日期为 2023-03 及之前
# A20250113184842199 channel_name为 顶部搜索框的搜索结果页， channel_name_cc 为顶部搜索框的搜索结果页，归属渠道 为抖音

dfzh1['是否免人审'] = np.where((dfzh1.is_vip=='1')&(dfzh1.status_result=='0'), 1 , 0)

# df['是否拒量'] = np.where(df.qvt_risk == '1', 1 , 0)
# df['是否拒量'] = np.where((df.qvt_risk == '0') & (df.qvt_result == '1'), 1 , df.是否拒量)
dfzh1['是否拒量'] = np.where(dfzh1.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
dfzh1['是否号卡'] = np.where(dfzh1.type == 4, 1, 0)

# df_zhys.columns
# df_zhys[['channel_name','channel_name_cc']]
# df_zhys.channel_name.value_counts()
# df_zhys.channel_name_cc.value_counts()
# 芝麻信用 芝麻租物 

## 1_14.渠道归属

In [79]:
# def qudao_type(a,b,c):
#     a = str(a)
#     b = str(b)
#     if "租物" in b:
#         return "芝麻租物"
#     elif "芝麻" in b:
#         return "芝麻租物"
#     elif "抖音" in b:
#         return "抖音渠道"
#     elif "搜索" in a:
#         return "搜索渠道"
#     elif "租物" in a:
#         return "芝麻租物"
#     elif "芝麻" in a:
#         return "芝麻租物"
#     elif c == 1:
#         return "芝麻租物" 
#     elif "叮咚直播" in a:
#         return "叮咚直播"
#     elif "八派信息" in a:
#         return "八派信息"
#     elif "租瓜直播2号" in a:
#         return "租瓜直播2号"
#     elif "租瓜直播" in a:
#         return "租瓜直播"
#     elif "直播" in a:
#         return "小舞直播"
#     elif "繁星" in a:
#         return "繁星"
#     elif "生活号" in a:
#         return "生活号"
#     elif "社群" in a:
#         return "支付宝社群"
#     elif "付费灯火" in a:
#         return "付费灯火"
#     else :
#         return a

In [80]:
# # df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
# dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
# dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

# dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# # pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [81]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [82]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    58561
二手     1248
Name: count, dtype: int64

In [83]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
# f_path_ck = "维客壹佰2023&2024年台账20250117.xlsx"
# path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/0_参数表/'
# dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# # dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
# dfck = dfck[[u'订单号',u'是否带锁']]
# dfck.shape
# dfck['是否带锁'].value_counts()

# dfck.drop_duplicates(subset=["订单号"],inplace=True)

# # 合并到主表中去
# # dfck 的带锁不一定出库，dfzh1 是出库的
# dfck = dfck.rename(columns= {'订单号': 'order_number'})
# dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
# dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
# dfzh1.shape

# pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

## 1_17.日期处理

In [84]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [85]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
2季度    20232
3季度    18894
4季度    11205
1季度     9478
Name: count, dtype: int64

In [90]:
dfzh1.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

## 1_21.获取_买断数据

In [86]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(7217, 7)

## 1_22.租后数据 关联 买断数据

In [87]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(59809, 90)

(59809, 86)

In [91]:
dfzh2.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

In [88]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(8837)

In [89]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      52915
已买断      6761
部分买断      133
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [18]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(3449, 7)

## 1_32.租后数据关联分期取消数据

In [19]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})
                                                        

(59809, 93)

(59809, 90)

In [92]:
dfzh3.columns

Index(['id', 'order_id', 'order_number', 'order_time', 'search_time',
       'end_date', 'merchant_id', 'merchant_name', 'channel_id',
       'channel_name', 'model_number', 'specification', 'purchase_amount',
       'platform_package', 'all_deposit', 'all_rental', 'liquidated_damages',
       'buyout_balance_payment', 'advance_periods', 'advance_sum',
       'order_create_time', 'rembursement_status', 'paid_periods',
       'begin_overdue_time', 'overdue_periods', 'max_overdue_days',
       'now_overdue_days', 'total_receivable', 'contract_price',
       'rent_received', 'unearned_rent_revenue', 'paid_buyout_balance_payment',
       'total_received', 'total_unreceived', 'unrecovered_cost',
       'overdue_type', 'gross_profit_receivable', 'overdue_loss',
       'actual_gross_profit', 'overdue_total_unreceived', 'relet_periods',
       'relet_days', 'relet_rent_paid', 'activity_id', 'activity_source',
       'order_label', 'create_time', 'update_time', 'order_finish_date',
       'comm

## 1_41.按风控策略调整时间节点划分时间段

In [20]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [21]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(789394, 23)

In [22]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(789394, 26)

(789394, 23)

In [23]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [93]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp
df_xzfq.columns

Index(['id', 'order_id', '当前应付租金', '实付金额', 'interest_money', '实付日期', 'status',
       '当前期数', 'order_pay_id', 'create_time', 'update_time', 'type',
       'advance_time', 'merchant_account_id', 'order_relet_id', 'is_relet',
       'pay_type', 'sesame_promo_money', 'sesame_promo_money_pay',
       'promo_money', 'plat_subsidy', 'promo_money_show', '应付日期'],
      dtype='object')

## 1_52.重命名

In [25]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [95]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')
df_xz_concat

Index(['id_x', 'order_id', '当前应付租金', '实付金额', 'interest_money', '实付日期',
       'status_x', '当前期数', 'order_pay_id', 'create_time_x',
       ...
       'mob_date_10', 'paid_date_mob10', 'paid_money_mob10', 'mob_date_11',
       'paid_date_mob11', 'paid_money_mob11', 'mob_date_12', 'paid_date_mob12',
       'paid_money_mob12', 'label'],
      dtype='object', length=171)

## 1_61.数据合并

In [27]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(789567)

(789567, 115)

(789394, 23)

(59809, 93)

## 1_62.去重

In [28]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
41780     431052   1.0
41792     431052   1.0
676364    431052   1.0
676376    431052   1.0
41781     431052   2.0
...          ...   ...
502610   2040621  23.0
449834   2040621  24.0
449858   2040621  24.0
502587   2040621  24.0
502611   2040621  24.0

[816 rows x 2 columns]


In [29]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(788955, 115)

In [30]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 3.计算逾期天数

## 3_11.异常查询

In [31]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 3_12.实还日期修正

In [32]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [33]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
496264,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
496265,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
496266,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
496267,A2025032020360980,2259696,666.32,666.32,2025-06-23,2025-06-23,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
496268,A2025032020360980,2259696,666.32,666.32,2025-07-23,2025-07-23,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT


In [34]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
496264,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
496265,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
496266,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
496267,A2025032020360980,2259696,666.32,666.32,2025-06-23,2025-06-23,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
496268,A2025032020360980,2259696,666.32,666.32,2025-07-23,2025-07-23,3.0,5.0,还款中,NaT,4,NaT,NaN,NaT


In [35]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60976,2024-07-22


In [36]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [37]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
658588,A20250621145050717,2978295,969.50,0.0,2026-05-24,NaT,1.0,12.0,已买断,2025-08-14 18:24:13,8,2025-08-14,11.0,2025-08-14
676962,A20250630102202280,3067220,1082.94,0.0,2026-05-03,NaT,1.0,11.0,已买断,2025-07-08 19:58:34,8,2025-07-08,10.0,2025-07-08
685867,A20250703102702367,3103364,1085.46,0.0,2026-06-06,NaT,1.0,12.0,已买断,2025-08-05 16:34:15,8,2025-08-05,11.0,2025-08-05
699426,A20250708114831820,3161546,1073.43,0.0,2026-05-11,NaT,1.0,11.0,已买断,2025-07-28 16:38:42,8,2025-07-28,10.0,2025-07-28
706134,A2025071014014127,3188905,1082.94,0.0,2026-05-13,NaT,1.0,11.0,已买断,2025-07-25 15:05:05,8,2025-07-25,10.0,2025-07-25


In [38]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [39]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [40]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [41]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [42]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [43]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [44]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [45]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 3_13.获取逾期截止时间

In [46]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-08-15
1   2025-08-15
2   2025-08-15
3   2025-08-15
4   2025-08-15
Name: now_date, dtype: datetime64[s]

## 3_14.重新定义状态

In [47]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    148837
已逾期                     82165
未到首期还款日                 59167
续租中                     20189
检测中                        92
Name: count, dtype: int64

订单状态 
租赁中      297490
已退款        8604
待归还        6033
待收货        4740
待发货        1976
已完成         221
检测中          89
订单取消         84
待商家收货         8
Name: count, dtype: int64

In [48]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     366894
未到还款日    306720
已逾期       53698
已逾期支付     52818
已取消        8825
Name: count, dtype: int64

In [49]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 3_15.重新定义逾期天数

In [50]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(319275)

Series([], Name: order_number, dtype: object)

In [51]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(788955, 119)

(788955, 119)

In [52]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        76635
已买断        11912
还款中        10781
续租中         5555
已完成         1404
检测中          161
未到首期还款日       55
Name: count, dtype: int64

In [53]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 3_16.逾期天数处理

In [54]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 3_17.稽查

In [55]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 3_18.确认是否到表现期

In [56]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [57]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [58]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [59]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [60]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-08-15,1
1,2022-09-04,2022-08-05,2025-08-15,1
2,2022-10-05,2022-09-05,2025-08-15,1
3,2022-11-04,2022-10-05,2025-08-15,1
4,2022-12-05,2022-11-05,2025-08-15,1


In [61]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 3_19.观察日定义 每月月底时间

In [62]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):
    # 2025.2.8 
    # for n in range(0,16):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat.shape
df_xz_concat = mob_date(df_xz_concat)
df_xz_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

(788955, 131)

(788955, 170)

In [63]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [64]:
# var = ['order_number','当前期数','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var]

## 3_20.更新逾期天数

In [65]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(1538, 2)

label
0.0    770619
1.0     18336
Name: count, dtype: int64

(1513, 171)

## 3_21.更新租期到期的逾期天数

In [66]:
# # （1）更新归还逾期的订单的逾期天数
# # 本次未统计在内，应该用 df_xz_concat

# # （1-1）未归还逾期或未到期或系统当天到期
# df_xz_concat.shape
# conditions_1 = (df_xz_concat.is_relet == 0)
# tmp1 = df_xz_concat[(conditions_1 & (df_xz_concat.end_date >= df_xz_concat.finish_date_new)) | (conditions_1 & (df_xz_concat.end_date >= df_xz_concat.date_str))]
# tmp1.shape

# # （1-2）续租
# tmp2 = df_xz_concat[df_xz_concat.is_relet > 0 ]
# tmp2.shape
# # （1-3）出现过归还逾期（包含租期一直逾期直到归还逾期的）--仅首续有归还逾期，被减数为最近未支付的应付日期
# conditions_2 = (df_xz_concat.end_date < df_xz_concat.finish_date_new)
# conditions_3 = (df_xz_concat.finish_date_new.isnull())
# conditions_4 = (df_xz_concat.end_date < df_xz_concat.date_str)
# tmp3 = df_xz_concat[(conditions_1 & conditions_2) | (conditions_1 & conditions_3 & conditions_4)]
# tmp3.shape

# # 367644 + 55490

# # 计算到期后逾期天数--订单已经完成的： 逾期天数为 finish_date_new- end_date；订单未完成的，逾期天数为 date_str- 实付日期new为空的最小的 应付日期
# # ((df_xz_concat['finish_date_new'] - df_xz_concat['end_date']).dt.days > 0) 
# # A2024020317504938--订单已完成
# # A202207171820512 --订单已完成
# # A202207181530165--订单未完成
# # var = ['order_number','当前期数','应付日期','实付日期new','end_date','date_str','逾期天数','pay_date','order_finish_date','订单完成时间','finish_date_new']
# # df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][var]

In [67]:
# （2）数据检查
# df_fq_concat[df_fq_concat.下单日期 == '2024-06-30'][['order_number']][:2]
# A2024060200290566  A2024063001194047 A202308262317036（续租） A2024031113200319（提早归还） A202207050851373（续租提前买断）
# var = ['order_number', '当前期数', '下单日期', '应付日期', '首期应还日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','是否提前还款','实付日期new','逾期天数','order_finish_date','finish_date_new']
# df_fq_concat[df_fq_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]

# var = ['order_number', '当前期数', '下单日期', '应付日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','实付日期new','finish_date_new','status_x','paid_periods','实付金额','date_tmp_1','date_tmp_2','订单状态_new','pay_date']
# var = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付金额','rembursement_status','订单状态','if_outpay','状态','首期应还日期', '买断价', '应付总租金', '买断尾款', '订单状态_new','租赁方案']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A2024031113200319'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202308262317036'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var] 

# 4.月末状态

## 4_1.月末非逾期类指标

In [96]:
df_xz_concat[:2]

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,update_time_x,type_x,advance_time,merchant_account_id,order_relet_id,is_relet,pay_type,sesame_promo_money,sesame_promo_money_pay,promo_money,plat_subsidy,promo_money_show,应付日期,id_y,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time_y,update_time_y,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type_y,order_type,tips,is_vip,status_result,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态,逾期天数,agr_1d,agr_4d,agr_7d,agr_15d,agr_30d,date_str,agr_1d_cum,agr_4d_cum,agr_7d_cum,agr_15d_cum,agr_30d_cum,mob_date_0,paid_date_mob0,paid_money_mob0,mob_date_1,paid_date_mob1,paid_money_mob1,mob_date_2,paid_date_mob2,paid_money_mob2,mob_date_3,paid_date_mob3,paid_money_mob3,mob_date_4,paid_date_mob4,paid_money_mob4,mob_date_5,paid_date_mob5,paid_money_mob5,mob_date_6,paid_date_mob6,paid_money_mob6,mob_date_7,paid_date_mob7,paid_money_mob7,mob_date_8,paid_date_mob8,paid_money_mob8,mob_date_9,paid_date_mob9,paid_money_mob9,mob_date_10,paid_date_mob10,paid_money_mob10,mob_date_11,paid_date_mob11,paid_money_mob11,mob_date_12,paid_date_mob12,paid_money_mob12,label
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,2022-07-05 10:41:10,0.0,NaT,6270.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-07-05,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11,301,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,未知渠道,未知渠道,0,0,0,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2025-08-15,正常还款,0,2022-07-06,2022-07-09,2022-07-12,2022-07-20,2022-08-04,2025-08-15,1,1,1,1,1,2022-07-31,2022-07-05,357.95,2022-08-31,2022-07-05,357.95,2022-09-30,2022-07-05,357.95,2022-10-31,2022-07-05,357.95,2022-11-30,2022-07-05,357.95,2022-12-31,2022-07-05,357.95,2023-01-31,2022-07-05,357.95,2023-02-28,2022-07-05,357.95,2023-03-31,2022-07-05,357.95,2023-04-30,2022-07-05,357.95,2023-05-31,2022-07-05,357.95,2023-06-30,2022-07-05,357.95,2023-07-31,2022-07-05,357.95,0.0
1,6507.0,8924,357.95,357.95,0.0,2022-08-05,3.0,2.0,9576,2022-07-05 08:51:37,2022-08-05 02:00:02,0.0,NaT,6277.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-08-05,1,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11,301,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,

In [69]:
# （1）
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1.loc[:,"租赁方案"]=np.where(df_xz_concat_1["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xz_concat_1['finish_date_new'] = pd.to_datetime(df_xz_concat_1['finish_date_new'])

tmp5 = df_xz_concat_1.groupby(['order_number']).agg({'应付日期':'max'}).reset_index()
tmp5 = tmp5.rename(columns = {'应付日期': '最大应还日期'})
df_xz_concat_1 = df_xz_concat_1.merge(tmp5, on = 'order_number', how = 'left')
df_xz_concat_1.shape

# （2）提取买断金等信息，区分续租和非续租 买断价，续租的买断价包含了首租的买断价
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租  'A202308262317036','A202207050851373'
sql = '''
SELECT tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期
    ,tod.actual_money 首租买断价
    ,tod.new_actual_money 续租买断价
    , tprm.all_deposit 总押金
    , tprm.total_receivable 总应收
    , tprm.total_received 总已收
    # , om.all_money 首租总租金
    # , (tod.new_actual_money-om.all_money) 续租总租金
    # , (tod.actual_money-om.all_money) 首租买断尾款
    , tprm.purchase_amount 采购金额
    , om.has_actual 是否买断
    , om.relet_status 是否续租
    , tprm.paid_periods 已还期数
    , tod.early_completion_mark 是否提前完成
    , tmu.id_card_num 身份证号
    , tol.delivery_province_name 地区
    , pa.type 活动类型
    , pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
'''

df_temp_1 = query(sql)
df_temp_1.shape
df_temp_1 = df_temp_1.drop_duplicates(subset = ['order_number'])
df_temp_1.shape

df_temp_1_1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_1[['order_number','首租买断价']], on = 'order_number', how = 'left')
df_temp_1_1 = df_temp_1_1.rename(columns = {'首租买断价': '买断价'})
df_temp_1_1.shape

df_temp_1_2 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_1[['order_number','续租买断价']], on = 'order_number', how = 'left')
df_temp_1_2 = df_temp_1_2.rename(columns = {'续租买断价': '买断价'})
df_temp_1_2.shape

df_xz_concat_1 = pd.concat([df_temp_1_1, df_temp_1_2])

# （3）加工应付总租金，区分续租和非续租 应付总租金
df_temp_2 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_2 = df_temp_2.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_2.shape

df_temp_3 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_3 = df_temp_3.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_3.shape

df_temp_4 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_2, on = 'order_number', how = 'left')
df_temp_4.shape

df_temp_5 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_3, on = 'order_number', how = 'left')
df_temp_5.shape

df_xz_concat_1 = pd.concat([df_temp_4, df_temp_5])


#（4）买断金 续租的买断尾款为 0 
df_xz_concat_1['买断尾款'] = df_xz_concat_1['买断价'] - df_xz_concat_1['应付总租金']
df_xz_concat_1['买断尾款'] = np.where(df_xz_concat_1.is_relet == 0, df_xz_concat_1['买断尾款'], 0)
df_xz_concat_1.shape

del df_temp_1_1
del df_temp_1_2
del df_temp_1
del df_temp_2
del df_temp_3
del df_temp_4
del df_temp_5

# （5）补数据  为了关联买断数据
# 当前期数、应付日期 改变，其他属性不变
tmp1 = df_xz_concat_1.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var = ['order_number','order_id','is_relet','当前期数', '下单日期', '应付日期', '买断价', '应付总租金', '买断尾款','租赁方案','rembursement_status','finish_date_new']
tmp1.shape

# tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp2['当前期数'] = tmp2['当前期数'] + 1
# tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = 1)

# tmp3 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp3['当前期数'] = tmp3['当前期数'] + 2
# tmp3['应付日期'] = tmp3['应付日期'] + pd.DateOffset(months = 2)

# tmp4 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
# tmp4['当前期数'] = tmp4['当前期数'] + 3
# tmp4['应付日期'] = tmp4['应付日期'] + pd.DateOffset(months = 3)

# tmp5 = pd.concat([tmp2, tmp3, tmp4])
# tmp5.shape

# 2025.6 取数，补够31期，补到2025.4月底

for num in range(1,31):
    tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
    tmp2['当前期数'] = tmp2['当前期数'] + num
    tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = num)
    df_xz_concat_1 = pd.concat([df_xz_concat_1, tmp2])

df_xz_concat_1.shape

# （6）订单状态定位到 当月
df_xz_concat_1['date_tmp_1'] = df_xz_concat_1['应付日期'].dt.strftime('%Y-%m')
df_xz_concat_1['date_tmp_1'] = pd.to_datetime(df_xz_concat_1['date_tmp_1'])
df_xz_concat_1['date_tmp_2'] = np.where(df_xz_concat_1['finish_date_new'].notnull(), pd.to_datetime(df_xz_concat_1['finish_date_new']).dt.strftime('%Y-%m'), np.nan)
df_xz_concat_1['date_tmp_2'] = pd.to_datetime(df_xz_concat_1['date_tmp_2'])

# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
df_xz_concat_1['订单状态_new'] = df_xz_concat_1.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已买断'
                                              else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已完成'
                                              else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                              else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                              else '其他', axis = 1
                                              )

df_xz_concat_1['提前状态'] = df_xz_concat_1.apply(lambda x: '提前买断' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已买断'
                                              else '已完成' if x.finish_date_new < x.最大应还日期 and x.订单状态_new == '已完成'
                                              else '其他', axis = 1
                                              )
# df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2'])
df_xz_concat_1.shape

# (7)关联买断数据 
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")
df_out_1['pay_date_m'] = df_out_1['pay_date'].dt.strftime('%Y-%m')
df_out_1['pay_date_m'] = pd.to_datetime(df_out_1['pay_date_m'])

df_out_2 = df_out_1.groupby(['order_id','pay_date_m']).agg({'real_pay_money':'sum'}).reset_index()
df_out_2 = df_out_2.rename(columns = {'real_pay_money': 'real_pay_money_new'})
df_out_2.shape

df_xz_concat_1 = df_xz_concat_1.merge(df_out_2[['order_id', 'real_pay_money_new', 'pay_date_m']], left_on = ['order_id', 'date_tmp_1'], right_on = ['order_id','pay_date_m'], how = 'left')

# 计算剩余买断尾款
df_xz_concat_1['real_pay_money_new'] = df_xz_concat_1['real_pay_money_new'].fillna(0)
df_xz_concat_1 = df_xz_concat_1.rename(columns = {'real_pay_money_new': '当月已收买断金'})
df_xz_concat_1 = df_xz_concat_1.sort_values(by = ['order_number','当前期数'])
df_xz_concat_1['当前累计已收买断金'] = df_xz_concat_1.groupby(['order_number'])['当月已收买断金'].cumsum()

df_xz_concat_1['剩余总应收买断金'] = df_xz_concat_1['买断尾款'] - df_xz_concat_1['当前累计已收买断金'] 
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['剩余总应收买断金'] < 0 , 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_xz_concat_1['剩余总应收买断金'])

# （8）2025.6.27 
# 更新实付金额：应付日期 大于 订单结束日期，实付金额为0的，且已买断的，用应付填充
conditions_1 = (df_xz_concat_1.应付日期 >= df_xz_concat_1.finish_date_new)
conditions_2 = (df_xz_concat_1.实付金额 == 0)
conditions_3 = (df_xz_concat_1.rembursement_status == '已买断')

df_xz_concat_1['实付金额'] = np.where(conditions_1 & conditions_2 & conditions_3, df_xz_concat_1.当前应付租金, df_xz_concat_1.实付金额)

df_xz_concat_1.shape
df_xz_concat_1[:1]

# var1 = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付日期new','实付金额','rembursement_status','订单状态','if_outpay','状态','订单状态_new']
# var2 = ['mob_date_11','paid_date_mob11','paid_money_mob11','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# var3 = ['order_id','当前期数','应付日期','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# var = var1 + var2 + var3

# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]
# 续租的实付买断金非0


(788955, 173)

(63691, 19)

(63664, 19)

(718810, 174)

(70145, 174)

...

(2582055, 180)

(6462, 3)

(6334, 3)

(2582055, 184)

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,update_time_x,type_x,advance_time,merchant_account_id,order_relet_id,is_relet,pay_type,sesame_promo_money,sesame_promo_money_pay,promo_money,plat_subsidy,promo_money_show,应付日期,id_y,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time_y,update_time_y,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type_y,order_type,tips,is_vip,status_result,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status_y,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,实付日期1,实付日期new,now_date,状态,逾期天数,agr_1d,agr_4d,agr_7d,agr_15d,agr_30d,date_str,agr_1d_cum,agr_4d_cum,agr_7d_cum,agr_15d_cum,agr_30d_cum,mob_date_0,paid_date_mob0,paid_money_mob0,mob_date_1,paid_date_mob1,paid_money_mob1,mob_date_2,paid_date_mob2,paid_money_mob2,mob_date_3,paid_date_mob3,paid_money_mob3,mob_date_4,paid_date_mob4,paid_money_mob4,mob_date_5,paid_date_mob5,paid_money_mob5,mob_date_6,paid_date_mob6,paid_money_mob6,mob_date_7,paid_date_mob7,paid_money_mob7,mob_date_8,paid_date_mob8,paid_money_mob8,mob_date_9,paid_date_mob9,paid_money_mob9,mob_date_10,paid_date_mob10,paid_money_mob10,mob_date_11,paid_date_mob11,paid_money_mob11,mob_date_12,paid_date_mob12,paid_money_mob12,label,租赁方案,最大应还日期,买断价,应付总租金,买断尾款,date_tmp_1,date_tmp_2,订单状态_new,提前状态,当月已收买断金,pay_date_m,当前累计已收买断金,剩余总应收买断金
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,2022-07-05 10:41:10,0.0,NaT,6270.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-07-05,1.0,A202207050851373,2022年07月,2022-07,2024-10-02,14.0,迪瓜优选,0.0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1.0,357.95,2022-07-05 08:51:37,已买断,12.0,NaT,0.0,4.0,0.0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11.0,301.0,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0.0,None,None,0.0,8.0,已完成,2024-04-17 01:37:33,0.0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,未知渠道,未知渠道,0.0,0.0,0.0,128G,金色,二手,2022-07-05,7.0,2022.0,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17,NaN,NaN,NaN,2022-07-05,2022-07-05,2025-08-15,正常还款,0.0,2022-07-06,2022-07-09,2022-07-12,2022-07-20,2022-08-04,2025-08-15,1.0,1.0,1.0,1.0,1.0,2022-07-31,2022-07-05,357.95,2022-08-31,2022-07-05,357.95,2022-09-30,2022-07-05,357.95,2022-10-31,2022-07-05,357.95,2022-11-30,2022-07-05,357.95,2022-12-31,2022-07-05,357.95,2023-01-31,2022-07-05,357.95,2023-02-28,2022-07-05,357.95,2023-03-31,2022-07-05,357.95,2023-04-30,2022-07-05,357.95,2023-05-31,2022-07-05,357.95,2023-06-30,2022-07-05,357.95,2023-07-31,2022-07-05,357.95,0.0,租完归还,2024-05-31,7530.0,4295.4,3234.6,2022-07-01,2024-04-01,还款中,其他,0.0,NaT,0.0,3234.6


## 4_2.首租

In [70]:
# （4-2）首租
# （1）非续租租期数据
# (384318, 161)
# df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]
# 2025.2.9
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_fq_concat = df_xz_concat_2[df_xz_concat_2.is_relet==0]
df_fq_concat['当前应付租金'] = np.where((df_fq_concat.订单状态_new == '已结束') & (df_fq_concat.实付金额 == 0), 0, df_fq_concat.当前应付租金)

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])
df_fq_concat.shape

# （2）12期加工
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# （2）
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    
    # 2025.2.11，当前最大期数不用 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
    df_mob_group_1_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前期数':'max'}).reset_index()
    
    df_mob_group_1 = df_mob_group_1.merge(df_mob_group_1_1, on = 'order_number', how = 'inner')

    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当前累计应付租金
df_mob_total = df_mob_total.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_total.tail(10)

df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape

# (520033, 170)
# (424063, 161)
# (384318, 163)
# (234387, 7)

# （3）关联多期的表
# 2025.2.9
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_total_tmp = df_xz_concat_1[df_xz_concat_1.is_relet == 0][var_1].merge(df_mob_total[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob','MOB']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet == 0].shape
df_mob_total_tmp.shape

tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
tmp_2.shape

df_mob_total_tmp = df_mob_total_tmp.merge(tmp_2, on = 'order_number', how ='inner')
df_mob_total_tmp.shape
del tmp_2

# 加工 df_mob_total_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
df_mob_total_tmp['订单状态_new'] = np.where((df_mob_total_tmp.is_relet_max >= 1) & (df_mob_total_tmp.当前期数 == 12), '已完成', df_mob_total_tmp['订单状态_new'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])
# 删去订单已结束的订单
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.订单状态_new != '已结束']
df_mob_total_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
# df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-01-31']
# df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= '2025-02-28']
# df_mob_total_tmp.shape

now_date = pd.Timestamp(dt.now().date())
df_mob_total_tmp = df_mob_total_tmp[df_mob_total_tmp.应付日期 <= now_date]
df_mob_total_tmp.shape

# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_total_tmp['overdue_days_mob'] = df_mob_total_tmp['overdue_days_mob'].fillna(0)
df_mob_total_tmp['当前累计应付租金'] = df_mob_total_tmp['当前累计应付租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_total.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_total[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 30, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 60, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + 90, df_mob_total_tmp['overdue_days_mob_x'])
# df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
# df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
# df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

for num in range(1,31):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_total_tmp = df_mob_total_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] > 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp['overdue_days_mob_y'] >= 0, df_mob_total_tmp['overdue_days_mob_y'] + num*30, df_mob_total_tmp['overdue_days_mob_x'])
    df_mob_total_tmp['当前累计应付租金'] = np.where(df_mob_total_tmp['当前累计应付租金_y'] > df_mob_total_tmp['当前累计应付租金_x'], df_mob_total_tmp['当前累计应付租金_y'], df_mob_total_tmp['当前累计应付租金_x'])
    df_mob_total_tmp['当前累计实付租金'] = np.where(df_mob_total_tmp['当前累计实付租金_y'] > df_mob_total_tmp['当前累计实付租金_x'], df_mob_total_tmp['当前累计实付租金_y'], df_mob_total_tmp['当前累计实付租金_x'])
    df_mob_total_tmp = df_mob_total_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})


df_mob_total_tmp.shape
df_mob_total_tmp['search_time'] = df_mob_total_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_total_tmp = df_mob_total_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_total_tmp.shape

df_mob_total_tmp['买断尾款'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.买断尾款)
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp.租赁方案 == '租完即送', 0, df_mob_total_tmp.剩余总应收买断金)

df_mob_total_tmp['应付总租金'] = df_mob_total_tmp['应付总租金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['剩余总应收租金'] = df_mob_total_tmp['应付总租金'] - df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['剩余总应收租金'] < 0, 0, df_mob_total_tmp['剩余总应收租金'])
df_mob_total_tmp['剩余总应收租金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收租金'])

df_mob_total_tmp['剩余总应收'] = df_mob_total_tmp['剩余总应收租金'] + df_mob_total_tmp['剩余总应收买断金']
df_mob_total_tmp['剩余总应收'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收'])
df_mob_total_tmp['剩余总应收买断金'] = np.where(df_mob_total_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_total_tmp['剩余总应收买断金'])

df_mob_total_tmp['当前累计已收买断金'] = df_mob_total_tmp['当前累计已收买断金'].fillna(0)
df_mob_total_tmp['当前累计实付租金'] = df_mob_total_tmp['当前累计实付租金'].fillna(0)
df_mob_total_tmp['当前累计实付'] = df_mob_total_tmp['当前累计已收买断金'] + df_mob_total_tmp['当前累计实付租金']
df_mob_total_tmp['总应收'] = df_mob_total_tmp['应付总租金'] + df_mob_total_tmp['买断尾款']

# A2023112717153031
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp['overdue_days_mob'] = np.where(df_mob_total_tmp.当前期数 == 1, 0 , df_mob_total_tmp.overdue_days_mob)
df_mob_total_tmp[:1]

# df_mob_total_tmp[df_mob_total_tmp.order_number == 'A2023112717153031'][['order_number','当前期数','应付日期','overdue_days_mob']]
# df_xz_concat[df_xz_concat.order_number == 'A2023112717153031'][['order_number','当前期数','应付日期','实付日期','实付金额','is_relet']]

df_mob_total_tmp['overdue_days_mob'] = np.where((df_mob_total_tmp.剩余总应收租金 == 0) & (df_mob_total_tmp.租赁方案 == '租完即送'), 0, df_mob_total_tmp.overdue_days_mob)

(718810, 172)

is_relet
0.0    718810
1.0     70074
2.0        71
Name: count, dtype: int64

(718810, 173)

(723422, 7)

(418967, 7)

...

(56419, 2)

(56419, 5)

(475965, 21)

(475965, 23)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,最大期数,剩余总应收租金,剩余总应收,当前累计实付,总应收
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,12.0,3937.45,7172.05,357.95,7530.0


## 4_3.续租

In [71]:
# （2-3）续租
# （1）续租数据
# (39745, 162)
# df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
# df_relet.shape

# # 2025.2.11
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_relet = df_xz_concat_2[df_xz_concat_2.is_relet >= 1]
df_relet['当前应付租金'] = np.where((df_relet.订单状态_new == '已结束') & (df_relet.实付金额 == 0), 0, df_relet.当前应付租金)

df_relet.shape
df_xz_concat.is_relet.value_counts()

# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet.shape
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                    #   2025.2.11
                    #   & ((df_relet['date_str'] - df_relet['应付日期_new']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum','当前期数':'max'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

# 2025.2.9
df_mob_relet.shape
df_mob_relet = df_mob_relet[df_mob_relet.当前应付租金 >= 0]
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_relet.shape
df_mob_relet = df_mob_relet.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_relet = df_mob_relet.drop_duplicates(subset = ['order_number','当前期数'])

df_mob_relet = df_mob_relet.rename(columns = {'当前应付租金': '当前累计应付租金', 'paid_money_mob': '当前累计实付租金'})
df_mob_relet.tail(10)

# （4）关联多期的表
# 2025.2.11
var_1 = ['order_number','下单日期','租赁方案','当前期数','应付日期', '当前应付租金', '实付金额','实付日期new','买断价','应付总租金','买断尾款','订单状态_new','当月已收买断金','当前累计已收买断金','剩余总应收买断金','提前状态']
df_mob_relet_tmp = df_xz_concat_1[df_xz_concat_1.is_relet >= 1][var_1].merge(df_mob_relet[['order_number','当前期数','当前累计应付租金','当前累计实付租金','overdue_days_mob','MOB']], on = ['order_number','当前期数'], how = 'left')
df_xz_concat_1[df_xz_concat_1.is_relet >= 1].shape
df_mob_relet_tmp.shape

# tmp_2 = df_xz_concat.groupby(['order_number']).agg({'is_relet':'max'}).reset_index()
# tmp_2 = tmp_2.rename(columns = {'is_relet': 'is_relet_max'})
# tmp_2.shape

# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_2, on = 'order_number', how ='inner')
# df_mob_relet_tmp.shape
# del tmp_2

# 加工 df_mob_relet_tmp
# 续租的订单只有12期，第12期的订单状态修改为已完成
# df_mob_relet_tmp['订单状态_new'] = np.where((df_mob_relet_tmp.is_relet_max >= 1) & (df_mob_relet_tmp.当前期数 == 12), '已完成', df_mob_relet_tmp['订单状态_new'])
# 删去订单已结束的订单
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.订单状态_new != '已结束']
df_mob_relet_tmp.shape
# 订单未结束的，删去未到期的，最新时间为 2025.2.10
# df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-01-31']
# df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= '2025-02-28']
df_mob_relet_tmp.shape

now_date = pd.Timestamp(dt.now().date())
df_mob_relet_tmp = df_mob_relet_tmp[df_mob_relet_tmp.应付日期 <= now_date]
df_mob_relet_tmp.shape

# 订单未结束的，到期未结束的则取完15期的数据，若租期一直逾期，则衍生到15期，若租期逾期，则默认不逾期
# A2024020317504938 2025.2 买断，还没表现出来
# 'A202207171820512' -- 9683，在15期之外买断，不加工租期逾期
# 'A202207181530165' --一直逾期，直至租期到了之后也逾期

df_mob_relet_tmp['overdue_days_mob'] = df_mob_relet_tmp['overdue_days_mob'].fillna(0)
df_mob_relet_tmp['当前累计应付租金'] = df_mob_relet_tmp['当前累计应付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)

tmp_2 = df_mob_relet.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp_2 = tmp_2.rename(columns = {'当前期数': '当前期数'})
tmp_3 = df_mob_relet[['order_number','当前期数','当前累计实付租金','当前累计应付租金','overdue_days_mob']].merge(tmp_2, on = ['order_number','当前期数'], how = 'inner')
tmp_2.shape
tmp_3.shape

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 30, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 60, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

# tmp_3['当前期数'] = tmp_3['当前期数'] + 1
# df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
# df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + 90, df_mob_relet_tmp['overdue_days_mob_x'])
# df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
# df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
# df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

for num in range(1,31):
    tmp_3['当前期数'] = tmp_3['当前期数'] + 1
    df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp_3, on = ['order_number','当前期数'], how = 'left')
    # df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] > 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp['overdue_days_mob_y'] >= 0, df_mob_relet_tmp['overdue_days_mob_y'] + num*30, df_mob_relet_tmp['overdue_days_mob_x'])
    df_mob_relet_tmp['当前累计应付租金'] = np.where(df_mob_relet_tmp['当前累计应付租金_y'] > df_mob_relet_tmp['当前累计应付租金_x'], df_mob_relet_tmp['当前累计应付租金_y'], df_mob_relet_tmp['当前累计应付租金_x'])
    df_mob_relet_tmp['当前累计实付租金'] = np.where(df_mob_relet_tmp['当前累计实付租金_y'] > df_mob_relet_tmp['当前累计实付租金_x'], df_mob_relet_tmp['当前累计实付租金_y'], df_mob_relet_tmp['当前累计实付租金_x'])
    df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = {'overdue_days_mob_x','overdue_days_mob_y','当前累计应付租金_x','当前累计应付租金_y','当前累计实付租金_x','当前累计实付租金_y'})

df_mob_relet_tmp.shape
df_mob_relet_tmp['search_time'] = df_mob_relet_tmp['下单日期'].dt.strftime('%Y-%m')

tmp = df_xz_concat[df_xz_concat.is_relet >= 1].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp1 = df_xz_concat[df_xz_concat.is_relet == 0].groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
tmp.shape
tmp1.shape
tmp = tmp.merge(tmp1, on = 'order_number', how = 'left')
tmp.shape
tmp['续租总期数'] = tmp['当前期数_x'] - tmp['当前期数_y']
tmp = tmp[['order_number','续租总期数']]

# tmp = tmp.rename(columns = {'当前期数': '最大期数'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp, on = 'order_number', how = 'left')
df_mob_relet_tmp.shape

# df_mob_relet_tmp['买断尾款'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.买断尾款)
# df_mob_relet_tmp['剩余总应收买断金'] = np.where(df_mob_relet_tmp.租赁方案 == '租完即送', 0, df_mob_relet_tmp.剩余总应收买断金)

df_mob_relet_tmp['应付总租金'] = df_mob_relet_tmp['应付总租金'].fillna(0)
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['剩余总应收租金'] = df_mob_relet_tmp['应付总租金'] - df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['剩余总应收租金'] < 0, 0, df_mob_relet_tmp['剩余总应收租金'])
df_mob_relet_tmp['剩余总应收租金'] = np.where(df_mob_relet_tmp['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_mob_relet_tmp['剩余总应收租金'])

# df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金'] + df_mob_relet_tmp['剩余总应收买断金']
df_mob_relet_tmp['剩余总应收'] = df_mob_relet_tmp['剩余总应收租金']
df_mob_relet_tmp['当前累计已收买断金'] = 0
df_mob_relet_tmp['当前累计实付租金'] = df_mob_relet_tmp['当前累计实付租金'].fillna(0)
df_mob_relet_tmp['当前累计实付'] = df_mob_relet_tmp['当前累计已收买断金'] + df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp['总应收'] = df_mob_relet_tmp['应付总租金'] + df_mob_relet_tmp['买断尾款']

# A2023112717153031
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.订单状态_new.isin(['已买断','已完成','已结束']), 0 , df_mob_relet_tmp.overdue_days_mob)
# note 2025.2.28
df_mob_relet_tmp['overdue_days_mob'] = np.where(df_mob_relet_tmp.剩余总应收租金 == 0, 0, df_mob_relet_tmp.overdue_days_mob)

df_mob_relet_tmp[:1]

(70145, 172)

is_relet
0.0    718810
1.0     70074
2.0        71
Name: count, dtype: int64

(70145, 208)

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


(46099, 7)

...

(6968, 2)

(59770, 2)

(6968, 3)

(45810, 22)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,续租总期数,剩余总应收租金,剩余总应收,当前累计实付,总应收
0,A202207050851373,2022-07-05,租完归还,13.0,2023-07-05,354.0,354.0,2023-07-07,8176.92,3953.0,0.0,还款中,0.0,0,0.0,其他,mob_12,0.0,354.0,354.0,2022-07,12.0,3599.0,3599.0,354.0,3953.0


## 4_4.表合并

In [105]:
df_mob_temp.columns


Index(['order_number', '下单日期', '租赁方案', '当前期数', '应付日期', '当前应付租金', '实付金额',
       '实付日期new', '买断价', '应付总租金', '买断尾款', '订单状态_new', '当月已收买断金', '当前累计已收买断金',
       '剩余总应收买断金', '提前状态', 'MOB', 'is_relet_max', 'overdue_days_mob',
       '当前累计应付租金', '当前累计实付租金', 'search_time', '剩余总应收租金', '剩余总应收', '当前累计实付',
       '总应收', '当前累计实付租金_all', '续租总期数', 'view_month', 'purchase_amount',
       'rembursement_status', '剩余采购成本', '最大期数', '最大期数_Bin', 'MOB_new',
       'MOB_new_1', '是否拒量', 'price', 'residue_money', 'unrecovered_cost',
       '剩余采购成本_new'],
      dtype='object')

In [72]:
df_mob_total_tmp.shape
df_mob_relet_tmp.shape
# view_month 为系统月的 overdue_days_mob 有误，不要使用

# 当前累计实付租金 等指标区分了首租和续租，要合并起来
tmp1 = df_mob_total_tmp.groupby(['order_number']).agg({'当前累计实付租金': 'max'}).reset_index()
tmp1 = tmp1.rename(columns = {'当前累计实付租金': '当前累计实付租金_首租'})
df_mob_relet_tmp = df_mob_relet_tmp.merge(tmp1, on = 'order_number', how = 'left')
df_mob_relet_tmp['当前累计实付租金_首租'] = df_mob_relet_tmp['当前累计实付租金_首租'].fillna(0)
df_mob_relet_tmp['当前累计实付租金_all'] = df_mob_relet_tmp['当前累计实付租金_首租'] + df_mob_relet_tmp['当前累计实付租金']
df_mob_relet_tmp = df_mob_relet_tmp.drop(columns = ['当前累计实付租金_首租'])

df_mob_total_tmp['当前累计实付租金_all'] = df_mob_total_tmp['当前累计实付租金']

df_mob_temp = pd.concat([df_mob_total_tmp, df_mob_relet_tmp])
# df_mob_temp['当前累计已收买断金'] = df_mob_temp['当前累计已收买断金'].fillna(0)
# df_mob_temp['当前累计实付租金'] = df_mob_temp['当前累计实付租金'].fillna(0)
# df_mob_temp['当前累计实付'] = df_mob_temp['当前累计已收买断金'] + df_mob_temp['当前累计实付租金']
# df_mob_temp['总应收'] = df_mob_temp['应付总租金'] + df_mob_temp['买断尾款']
df_mob_temp.shape
# df_mob_temp[:1]


# 更新续租订单的订单状态：（只更新首租到期的那期的订单状态） 
# 订单状态_new 为各期对应的状态；rembursement_status为订单当前状态
df_mob_temp['订单状态_new'] = np.where((df_mob_temp['订单状态_new'] == '已完成') & (df_mob_temp.is_relet_max >= 1), '还款中', df_mob_temp['订单状态_new'])
df_mob_temp['view_month'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')


# 计算剩余采购成本
# 实付金额全部折为采购金额， 若剩余采购金额<0，则为0 
# 已买断/已完成/检测中 的当月及之后，剩余采购成本为0
# 检测中的订单，实付日期为空的最小的应付日期那条，剩余采购成本为0，逾期天数为0,大于最小应付日期的 样本删去

df_mob_temp = df_mob_temp.merge(df_xz_concat[df_xz_concat.当前期数 == 1][['order_number','purchase_amount','rembursement_status']], on = 'order_number', how = 'left')

df_mob_temp['剩余采购成本'] = df_mob_temp['purchase_amount'] - df_mob_temp['当前累计实付租金_all']
# 2025.7.4 修改：不跑，统一在 统计 的前一步跑
# df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.订单状态_new.isin(['已买断','已完成']), 0, df_mob_temp.剩余采购成本)

tmp1 = df_mob_temp[(df_mob_temp.rembursement_status == '检测中') & (df_mob_temp.实付日期new.isnull())].groupby(['order_number']).agg({'应付日期': 'min'}).reset_index()
tmp1 = tmp1.rename(columns = {'应付日期': '应付日期_检测中'})
df_mob_temp = df_mob_temp.merge(tmp1, on = ['order_number'], how = 'left')
df_mob_temp['剩余采购成本'] = np.where((df_mob_temp.rembursement_status == '检测中')&(df_mob_temp.应付日期 >= df_mob_temp.应付日期_检测中), 0, df_mob_temp.剩余采购成本)
df_mob_temp['overdue_days_mob'] = np.where((df_mob_temp.rembursement_status == '检测中')&(df_mob_temp.应付日期 >= df_mob_temp.应付日期_检测中), 0, df_mob_temp.overdue_days_mob)

tmp1 = df_mob_temp[df_mob_temp.rembursement_status != '检测中']
tmp2 = df_mob_temp[df_mob_temp.rembursement_status == '检测中']
tmp2 = tmp2[tmp2.应付日期 <= tmp2.应付日期_检测中]
df_mob_temp = pd.concat([tmp1, tmp2])
df_mob_temp = df_mob_temp.drop(columns = ['应付日期_检测中','最大期数'])

tmp1 = df_xz_concat.groupby(['order_number']).agg({'当前期数': 'max'}).reset_index()
tmp1 = tmp1.rename(columns = {'当前期数': '最大期数'})
df_mob_temp = df_mob_temp.merge(tmp1, on = ['order_number'], how = 'left')
df_mob_temp['最大期数_Bin'] = df_mob_temp.apply(lambda x: '12个月' if x.最大期数 <= 12
                                            else '18个月' if x.最大期数 > 12 and x.最大期数 <= 18
                                            else '24个月' if x.最大期数 > 18
                                            else 'other', axis = 1)

# 更新MOB -方式1（弱点：# 差异对比 如 A2023122617252850：view_month == '2024-12' 没有，因为续租跳月了，但MOB12有）
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.view_month == '2024-12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.MOB_new_1 == 'MOB12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()
# df_mob_temp[(~df_mob_temp.order_number.isin(not_count_order_number))&(df_mob_temp.MOB_new == 'MOB12')&(df_mob_temp.search_time == '2023-12')].剩余采购成本.sum()

df_mob_temp['date_tmp_1'] = df_mob_temp['应付日期'].dt.strftime('%Y-%m')
df_mob_temp['date_tmp_1'] = pd.to_datetime(df_mob_temp['date_tmp_1'])
df_mob_temp['date_tmp_2'] = df_mob_temp['下单日期'].dt.strftime('%Y-%m')
df_mob_temp['date_tmp_2'] = pd.to_datetime(df_mob_temp['date_tmp_2'])

df_mob_temp['当前期数_tmp'] = df_mob_temp.apply(lambda x: 1 if x.date_tmp_1 == x.date_tmp_2 and x.当前期数 == 1
                                            else 0 if x.date_tmp_1 != x.date_tmp_2 and x.当前期数 == 1
                                            else np.nan, axis = 1)
# 向下填充
df_mob_temp['当前期数_tmp'] = df_mob_temp.groupby('order_number')['当前期数_tmp'].ffill()
df_mob_temp['当前期数_MOB_2'] = df_mob_temp['当前期数'] - df_mob_temp['当前期数_tmp']
df_mob_temp['当前期数_MOB_2'] = df_mob_temp['当前期数_MOB_2'].astype('str')
df_mob_temp['当前期数_MOB_1'] = 'MOB'
df_mob_temp['MOB_new'] = df_mob_temp['当前期数_MOB_1'] + df_mob_temp['当前期数_MOB_2']
df_mob_temp['MOB_new'] = df_mob_temp['MOB_new'].str.replace('.0','')

df_mob_temp = df_mob_temp.drop(columns = ['date_tmp_1','date_tmp_2','当前期数_tmp','当前期数_MOB_1','当前期数_MOB_2'])


# 更新MOB -方式2
df_mob_temp['下单日期_year'] = pd.to_datetime(df_mob_temp['下单日期']).dt.year
df_mob_temp['下单日期_month'] = pd.to_datetime(df_mob_temp['下单日期']).dt.month

df_mob_temp['应付日期_year'] = pd.to_datetime(df_mob_temp['应付日期']).dt.year
df_mob_temp['应付日期_month'] = pd.to_datetime(df_mob_temp['应付日期']).dt.month

df_mob_temp['下单日期_year'] = df_mob_temp['下单日期_year'].astype(float)
df_mob_temp['下单日期_month'] = df_mob_temp['下单日期_month'].astype(float)
df_mob_temp['应付日期_year'] = df_mob_temp['应付日期_year'].astype(float)
df_mob_temp['应付日期_month'] = df_mob_temp['应付日期_month'].astype(float)


df_mob_temp['mob_numbers'] = np.where(pd.notnull(df_mob_temp.应付日期), (df_mob_temp.应付日期_year - df_mob_temp.下单日期_year)*12 + df_mob_temp.应付日期_month - df_mob_temp.下单日期_month, -999)
df_mob_temp['mob_numbers'] = df_mob_temp['mob_numbers'].astype('str')

df_mob_temp['当前期数_MOB_1'] = 'MOB'
df_mob_temp['MOB_new_1'] = df_mob_temp['当前期数_MOB_1'] + df_mob_temp['mob_numbers']
df_mob_temp['MOB_new_1'] = df_mob_temp['MOB_new_1'].str.replace('.0','')

df_mob_temp = df_mob_temp.drop(columns = ['下单日期_year','下单日期_month','应付日期_year','应付日期_month','mob_numbers','当前期数_MOB_1'])



df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','当前应付租金','实付金额','当前期数','当前累计实付租金',
#          '应付日期','当前累计实付租金_all','purchase_amount','剩余采购成本','实付日期new','overdue_days_mob','订单状态_new','rembursement_status']
# df_mob_temp[df_mob_temp.order_number.isin(['A202207050851373'])][var_1] # 续租，续租后买断
# df_mob_temp[df_mob_temp.order_number.isin(['A202207131230485'])][var_1] # 当月下单当月完成
# df_mob_temp[df_mob_temp.order_number.isin(['A202506261458391158'])][var_1] # 当月下单当月完成
# df_mob_temp[df_mob_temp.order_number.isin(['A202208231129327'])][var_1] # 检测中	,1+1
# df_mob_temp[df_mob_temp.order_number.isin(['A20240617115834268'])][var_1] # 检测中	,1+2
# df_mob_temp[df_mob_temp.order_number.isin(['A20240430114158109'])][var_1] # 订单状态_new == '已买断' 则 剩余采购成本 = 0 A20240430114158109（月末订单）/A2025051810143580（已完成） 也是

# var_1 = ['order_number','order_id','search_time','is_relet','当前应付租金','实付金额','应付日期','当前期数','实付日期','finish_date_new','rembursement_status','purchase_amount']
# df_xz_concat_1[df_xz_concat_1.order_number == 'A202207050851373'][var_1]

(475965, 27)

(45810, 26)

(521775, 29)

(520777, 36)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0


## 4_5.碎屏险/押金

In [73]:
# （1）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o[df_s_o.price > 0].shape
# df_s_o[(df_s_o.price > 0)&(df_s_o.special_price > 0)].shape
# df_s_o[(df_s_o.price < df_s_o.special_price)].shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape
# df_s_o[(df_s_o.price>0)&(df_s_o.special_price>0)].tail()

dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

# （2）溢价费 -- 未回收成本中已经减掉了 溢价费
# sql = ''' 
# SELECT  distinct product_id,commodity_premium
# from db_digua_business.t_product_sku
# '''
# df_p_s = query(sql)
# df_p_s.shape
# count(*)	count(distinct product_id)
# 	319176	6868
# df_p_s.product_id.value_counts()
# df_p_s[df_p_s.product_id == 4321]

# dfzh3['commodity_premium'] = dfzh3['commodity_premium'].fillna(0)

# （3）押金
# 2025.3.3
# `residue_money` double(10,2) DEFAULT NULL COMMENT '剩余未使用押金金额'
sql = ''' 
SELECT  distinct order_id,residue_money
from db_digua_business.t_order_pledge
where residue_money > 0
'''
df_p = query(sql)
df_p.shape
df_p.drop_duplicates(subset = ['order_id']).shape

dfzh3 = dfzh3.merge(df_p, on = 'order_id', how = 'left')
dfzh3['residue_money'] = dfzh3['residue_money'].fillna(0)
dfzh3.shape
dfzh3[:1]

(307506, 2)

(307506, 2)

(305230, 2)

(59809, 93)

(59809, 94)

(3184, 2)

(3184, 2)

(59809, 95)

,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,订单状态值,订单状态,订单完成时间,order_method,true_name,id_card_num,mobile,province_name,product_name,sku_attributes,channel_name_cc,channel_type_id,activity_name,type,order_type,tips,is_vip,status_result,来源渠道,归属渠道,是否免人审,是否拒量,是否号卡,内存,颜色,是否二手,下单日期,月份,年份,季度,outpay_money,pay_date,status,if_outpay,finish_date_new,nn,reality_refund_date,real_pay_money,price,residue_money
0,1,8924,A202207050851373,2022年07月,2022-07,2024-10-02,14,迪瓜优选,0,未知渠道,99新 iPhone 12 Pro 国行 官方质检【租物特惠】,内存：128G 颜色：金色,5150.0,租完归还/可买断,5303.0,4295.4,0.0,2121.2,1,357.95,2022-07-05 08:51:37,已买断,12,NaT,0,4,0,6416.6,7530.0,7847.2,-3881.52,329.72,8176.92,-1760.32,-3026.92,,1266.6,0.0,3026.92,0.0,11,301,3551.8,NaN,None,,2024-07-16 11:27:01,2024-07-18 15:50:56,2024-04-17 01:37:33,0.0,0.0,0,None,None,0,8,已完成,2024-04-17 01:37:33,0,胡水飘,362202199805282550,13027948681,江西省,官方质检 iPhone 12 Pro 国行 99新,"[{""key"":""内存"",""value"":""128G""},{""key"":""颜色"",""valu...",None,NaN,None,NaN,ZFB_ORDER,None,None,None,未知渠道,未知渠道,0,0,0,128G,金色,二手,2022-07-05,7,2022,3季度,329.72,2024-04-17 01:37:33,2.0,已买断,2024-04-17 01:37:33,NaN,NaN,NaN,0.0,0.0


In [74]:
# （4）
dfzh3 = dfzh3.drop_duplicates(subset = ['order_number'])
# unrecovered_cost 未回收成本--截面的数据

df_mob_temp = df_mob_temp.merge(dfzh3[['order_number','是否拒量','price','residue_money','unrecovered_cost']], on = 'order_number', how = 'left')
df_mob_temp['residue_money'] = df_mob_temp['residue_money'].fillna(0)
df_mob_temp['price'] = df_mob_temp['price'].fillna(0)

# 更新剩余采购成本
# 扣去碎屏险
df_mob_temp['剩余采购成本'] = df_mob_temp['剩余采购成本'] - df_mob_temp['price']
# 逾期的扣去押金
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.overdue_days_mob > 0, df_mob_temp.剩余采购成本 - df_mob_temp.residue_money, df_mob_temp.剩余采购成本)

# 2025.7.4 修改：不跑，统一在 统计 的前一步跑
# df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)
df_mob_temp.shape
df_mob_temp[:1]

# var_1 = ['order_number','purchase_amount','当前累计实付租金_all','剩余采购成本','当前期数','应付日期','price','residue_money','unrecovered_cost']
# df_mob_temp[df_mob_temp.order_number == 'A20250331235111137'][var_1]
# df_mob_temp[df_mob_temp.order_number == 'A20250321113738291'][var_1]

(520777, 40)

,order_number,下单日期,租赁方案,当前期数,应付日期,当前应付租金,实付金额,实付日期new,买断价,应付总租金,买断尾款,订单状态_new,当月已收买断金,当前累计已收买断金,剩余总应收买断金,提前状态,MOB,is_relet_max,overdue_days_mob,当前累计应付租金,当前累计实付租金,search_time,剩余总应收租金,剩余总应收,当前累计实付,总应收,当前累计实付租金_all,续租总期数,view_month,purchase_amount,rembursement_status,剩余采购成本,最大期数,最大期数_Bin,MOB_new,MOB_new_1,是否拒量,price,residue_money,unrecovered_cost
0,A202207050851373,2022-07-05,租完归还,1.0,2022-07-05,357.95,357.95,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,0.0,3234.6,其他,mob_0,2.0,0.0,357.95,357.95,2022-07,3937.45,7172.05,357.95,7530.0,357.95,NaN,2022-07,5150.0,已买断,4792.05,24.0,24个月,MOB0,MOB0,0,0.0,0.0,-3026.92


## 4_6.灵活调整

In [75]:
# （1）剩余采购成本 调整
# 这个口径放在这里统一跑，灵活调整
df_mob_temp['剩余采购成本_new'] = df_mob_temp['剩余采购成本']
df_mob_temp['剩余采购成本'] = np.where(df_mob_temp.剩余采购成本 < 0, 0 , df_mob_temp.剩余采购成本)

df_mob_temp_new = df_mob_temp.copy()
# df_goal_temp = df_mob_temp.copy()

In [104]:
tmp = df_mob_temp[df_mob_temp.view_month == '2025-07']
# 
tmp = tmp.sort_values(by=['下单日期'], ascending=False)
tmp.下单日期

470321   2025-07-28
470316   2025-07-28
470327   2025-07-28
470326   2025-07-28
470325   2025-07-28
            ...    
473550   2022-07-27
272      2022-07-20
198      2022-07-20
235      2022-07-20
148      2022-07-19
Name: 下单日期, Length: 49271, dtype: datetime64[ns]

In [76]:
df_mob_temp.shape
# df_mob_temp[:1]

# df_mob_temp[df_mob_temp.search_time == '2025-07'].总应收.sum()
df_mob_temp[df_mob_temp.view_month == '2025-07'].买断价.sum()
df_mob_temp[df_mob_temp.view_month == '2025-07'].purchase_amount.sum()
df_mob_temp[df_mob_temp.view_month == '2025-07'].应付总租金.sum()

df_mob_temp[df_mob_temp.view_month == '2025-07'].剩余总应收.sum()
df_mob_temp[df_mob_temp.view_month == '2025-07'].剩余总应收租金.sum()

(520777, 41)

np.float64(597539983.1400001)

np.float64(390238696.15)

np.float64(325516950.08000004)

np.float64(409981521.86)

np.float64(175936959.60999998)